In [1]:
#Step1. Data Stock : Crawling Naver "https://finance.naver.com/"
# - BrautifulSoup - Request - Dataframe
#어떠한 정보를 Gathering 할 것 인가. 
# 거래량, 외인 거래량, 기관 거래량, 시가(전날 종가가 시가인데 의미가있나?), 종가 ?
#Step2. Data Gathering : CSV, DB?
# - 
#Step3. Model
# - 분류를 어떻게 나눌것인가 (Classifier : 종가 상승인지 하강인지? or 5분 뒤 상승인지 하강인지?)
# - Model : Random Forest?

In [2]:
# 불필요한 경고 메시지 끄기
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import font_manager
import requests
from openpyxl import Workbook
from io import BytesIO
from bs4 import BeautifulSoup
import re

In [4]:
custom_headers = {
    "referer":'https://naver.com/',
    "user-agent":'Mozilla'
}

In [5]:
wb = Workbook()
ws = wb.active
ws.append(["일자","종가","거래량","기관순매수","기관순매매","보유율"])
code = "034730"
max_page = 100

# code = input("종목 코드 : ")
# max_page = input("수집할 페이지 수 : ")

In [6]:
if max_page:
    max_page = int(max_page)
else:
    max_page = 0

In [7]:
data = []

In [8]:
for myPage in range(1,max_page):
    url = f"https://finance.naver.com/item/frgn.naver?code={code}&page={myPage}"
    res = requests.get(url, headers=custom_headers)
    soup = BeautifulSoup(res.text,"lxml")
    pageTable = soup.find("table", attrs={ "class" : "Nnavi", "summary":"페이지 네비게이션 리스트"})
    if int(pageTable.find("td",attrs = {"class":"on"}).text.strip()) != myPage:
        break
    table = soup.find("table", attrs={ "class" : "type2", "summary":"외국인 기관 순매매 거래량에 관한표이며 날짜별로 정보를 제공합니다."})
    for idx, dataraw in enumerate(table.findAll("tr")):
        if idx <= 2:
            continue
        cols = dataraw.findAll("td")
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele]) 
    

In [9]:
df = pd.DataFrame(data)

In [10]:
dscolumnls = ["날짜","종가","전일비","등락률","거래량","순매매량","순매매-량","외인보유주수","외인보유율"]

In [11]:
df.columns = dscolumnls

In [12]:
df = df.dropna()

In [13]:
df.to_csv('SK_StockData.csv', sep=',', na_rep='NaN')

In [14]:
#Data Fram중에 필요한 정보를 Collection 한다 
#Data Frame 의 모든 String 을 Int or Date 등등으로 변경한다.
#그래프도 한번 그려보자
#포인트도 짝어보자
#모델은 뭐쓰지?

In [15]:
df['날짜'] = pd.to_datetime(df['날짜'])

In [16]:
df.set_index('날짜', inplace=True)

In [17]:
dfordered = df

In [18]:
dfordered

,종가,전일비,등락률,거래량,순매매량,순매매-량,외인보유주수,외인보유율
날짜,,,,,,,,
2022-03-16,"234,500","1,000",+0.43%,"111,860",-378,"-11,251","15,959,044",21.52%
2022-03-15,"233,500","4,000",-1.68%,"96,304",+913,"-9,504","15,970,295",21.54%
2022-03-14,"237,500","9,500",-3.85%,"156,359","+9,757","-9,996","15,966,861",21.53%
2022-03-11,"247,000","10,500",+4.44%,"304,725","+138,876","-61,186","15,952,165",21.51%
2022-03-10,"236,500","5,500",+2.38%,"197,274","+83,292","-48,073","16,029,527",21.62%
...,...,...,...,...,...,...,...,...
2014-03-03,"145,000",0,0.00%,"96,548","-16,464","-5,720","9,625,350",19.25%
2014-02-28,"145,000","2,500",+1.75%,"215,891","+14,791","-1,187","9,621,470",19.24%
2014-02-27,"142,500","4,500",+3.26%,"228,818","+45,341","+11,216","9,613,105",19.23%


In [19]:
#df 정보 확인
dfordered.describe()

,종가,전일비,등락률,거래량,순매매량,순매매-량,외인보유주수,외인보유율
count,1980,1980,1980,1980,1980,1980,1980,1980
unique,346,49,707,1971,1969,1950,1980,671
top,"270,000","1,000",0.00%,"85,899","+13,894","+6,922","15,959,044",27.62%
freq,24,203,114,2,2,3,1,13


In [20]:
#df 누락 여부 확인, object 항목들에 data 화 필요
#전날 대비 종가 상승인지 하락인지 확인하는 추가 Column 필요?????? 그게 맞나?
#5,10,20,30일선 추가
#전일비, 등락률 Column 삭제
dfordered.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1980 entries, 2022-03-16 to 2014-02-25
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   종가      1980 non-null   object
 1   전일비     1980 non-null   object
 2   등락률     1980 non-null   object
 3   거래량     1980 non-null   object
 4   순매매량    1980 non-null   object
 5   순매매-량   1980 non-null   object
 6   외인보유주수  1980 non-null   object
 7   외인보유율   1980 non-null   object
dtypes: object(8)
memory usage: 139.2+ KB


In [22]:
dfordered.drop(["전일비","등락률","외인보유주수"],axis="columns",inplace = True)

In [23]:
dfordered.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1980 entries, 2022-03-16 to 2014-02-25
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   종가      1980 non-null   object
 1   거래량     1980 non-null   object
 2   순매매량    1980 non-null   object
 3   순매매-량   1980 non-null   object
 4   외인보유율   1980 non-null   object
dtypes: object(5)
memory usage: 92.8+ KB


In [24]:
dfordered["종가"] = dfordered["종가"].str.replace(",","")
dfordered["종가"] = dfordered["종가"].astype(float)
dfordered["거래량"] = dfordered["거래량"].str.replace(",","")
dfordered["거래량"] = dfordered["거래량"].astype(float)
dfordered["순매매량"] = dfordered["순매매량"].str.replace(",","")
dfordered["순매매량"] = dfordered["순매매량"].astype(float)
dfordered["순매매-량"] = dfordered["순매매-량"].str.replace(",","")
dfordered["순매매-량"] = dfordered["순매매-량"].astype(float)
dfordered["외인보유율"] = dfordered["외인보유율"].str.replace(",","")
dfordered["외인보유율"] = dfordered["외인보유율"].str.replace("%","")
dfordered["외인보유율"] = dfordered["외인보유율"].astype(float)

In [26]:
dfordered.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1980 entries, 2022-03-16 to 2014-02-25
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   종가      1980 non-null   float64
 1   거래량     1980 non-null   float64
 2   순매매량    1980 non-null   float64
 3   순매매-량   1980 non-null   float64
 4   외인보유율   1980 non-null   float64
dtypes: float64(5)
memory usage: 92.8 KB


In [27]:

dfordered['MA5'] = dfordered['종가'].rolling(window=5).mean()
# 10, 20, 30, 60
dfordered['MA10'] = dfordered['종가'].rolling(window=10).mean()
dfordered['MA20'] = dfordered['종가'].rolling(window=20).mean()
dfordered['MA30'] = dfordered['종가'].rolling(window=30).mean()
dfordered['MA60'] = dfordered['종가'].rolling(window=60).mean()

In [28]:
dfordered.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1980 entries, 2022-03-16 to 2014-02-25
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   종가      1980 non-null   float64
 1   거래량     1980 non-null   float64
 2   순매매량    1980 non-null   float64
 3   순매매-량   1980 non-null   float64
 4   외인보유율   1980 non-null   float64
 5   MA5     1976 non-null   float64
 6   MA10    1971 non-null   float64
 7   MA20    1961 non-null   float64
 8   MA30    1951 non-null   float64
 9   MA60    1921 non-null   float64
dtypes: float64(10)
memory usage: 170.2 KB


chan test